## Introduction

In this assignment I'll be using the following scenario: A boutique hotel in Madrid has decided to expand it's precence to Paris and wants to open up a new hotel in central Paris.

The hotel wants to make sure that there are similar characteristics in the neighbourhood in Paris compared to its central Madrid location, so that the hotel will attract customers who look for similar experiences.

In order to find out which area satisfies these requirements, the central areas of the two cities will be compared using K-Means clustering technique.

## Data

This analysis will use data obtained mainly from Geopy, Wikipedia and Foursquare. 


In [1]:
# importing required modules
import pandas as pd
import requests


In [2]:
# importing Madrid neighborhoods
url = 'https://en.wikipedia.org/wiki/List_of_wards_of_Madrid'
results = requests.get(url).text
df = pd.read_html(results, header=0, attrs={"class":"wikitable sortable"})[0]
df = df[0:128]

df.drop(['Number','Image'],axis=1,inplace=True)

df.rename(columns={'Name':'Neighborhood'},inplace=True)
df = df.groupby(['District'])['Neighborhood'].apply(','.join).reset_index()

df_raw = df.copy()
df_raw.head()

df.head()

df_rev = df_raw.set_index('District').Neighborhood.str.split(',', expand=True).stack().reset_index('District')
df_rev.rename(columns={0:'Neighborhood'},inplace=True)
df_rev.Neighborhood = df_rev.Neighborhood.astype(str) + ', Madrid, ES'
mad = df_rev.copy()
mad.reset_index(drop=True, inplace=True)

mad

# # filtering only the area of the hotel in Madrid
mad = mad.loc[mad['Neighborhood'] == 'Atocha, Madrid, ES']
mad.reset_index(drop=True, inplace=True)
mad

,District,Neighborhood
0,Arganzuela,"Atocha, Madrid, ES"


In [50]:
# importing Paris neighborhoods
url = 'https://en.wikipedia.org/wiki/Arrondissements_of_Paris#Arrondissements'
results = requests.get(url).text
df = pd.read_html(results, header=0, attrs={"class":"wikitable sortable"})[0]

df.drop(['Area (km2)','Population(March 1999 census)','Population(July 2005 estimate)','Density (2005)(inhabitants per km2)','Peak of population','Mayor'],
        axis=1,inplace=True)
df.rename(columns={'Arrondissement (R for Right Bank, L for Left Bank)':'District','Name':'Neighborhood'},inplace=True)

df.Neighborhood = df.Neighborhood.astype(str) + ', Paris, FR'
pa = df.copy()

pa

,District,Neighborhood
0,1st (Ie) R,"Louvre, Paris, FR"
1,2nd (IIe) R,"Bourse, Paris, FR"
2,3rd (IIIe) R,"Temple, Paris, FR"
3,4th (IVe) R,"Hôtel-de-Ville, Paris, FR"
4,5th (Ve) L,"Panthéon, Paris, FR"
5,6th (VIe) L,"Luxembourg, Paris, FR"
6,7th (VIIe) L,"Palais-Bourbon, Paris, FR"
7,8th (VIIIe) R,"Élysée, Paris, FR"
8,9th (IXe) R,"Opéra, Paris, FR"
9,10th (Xe) R,"Entrepôt, Paris, FR"


In [51]:
# merging dataframes for next steps
df2 = mad.append(pa)
df2.reset_index(drop=True, inplace=True)
df2

,District,Neighborhood
0,Arganzuela,"Atocha, Madrid, ES"
1,1st (Ie) R,"Louvre, Paris, FR"
2,2nd (IIe) R,"Bourse, Paris, FR"
3,3rd (IIIe) R,"Temple, Paris, FR"
4,4th (IVe) R,"Hôtel-de-Ville, Paris, FR"
5,5th (Ve) L,"Panthéon, Paris, FR"
6,6th (VIe) L,"Luxembourg, Paris, FR"
7,7th (VIIe) L,"Palais-Bourbon, Paris, FR"
8,8th (VIIIe) R,"Élysée, Paris, FR"
9,9th (IXe) R,"Opéra, Paris, FR"


## Analysis

In [5]:
#importing modules
import numpy as np
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [6]:
# @hidden_cell
CLIENT_ID = 'useyourown' # your Foursquare ID
CLIENT_SECRET = 'useyourown' # your Foursquare Secret
VERSION = '20190112' # Foursquare API version

radius = 500
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AUTNE4CTPVEQP0SXXX2TI4IFZQJDFXCVOGBZRC5REXQC3ZRU
CLIENT_SECRET:JXEQ12SHW4PYAVXNVNR10FW04VPCCSCSK3BQCTUIKAEMZXSQ


In [7]:
df2.shape


(10, 2)

In [17]:
def findlat(row):
    geolocator = Nominatim(user_agent="my-app")
    location = geolocator.geocode(row['Neighborhood'], timeout=10)
    lat = location.latitude
    return lat

In [18]:
def findlng(row):
    geolocator = Nominatim(user_agent="my-app")
    location = geolocator.geocode(row['Neighborhood'], timeout=10)
    lng = location.longitude
    return lng

In [52]:
# finding latitude and longitude per each neighborhood in dataset
df2['lat'] = df2.apply(findlat, axis=1)
df2['lng'] = df2.apply(findlng, axis=1)
print(df2.shape)
df2.head()

(21, 4)


,District,Neighborhood,lat,lng
0,Arganzuela,"Atocha, Madrid, ES",40.400687,-3.682391
1,1st (Ie) R,"Louvre, Paris, FR",48.861147,2.338028
2,2nd (IIe) R,"Bourse, Paris, FR",48.867684,2.343126
3,3rd (IIIe) R,"Temple, Paris, FR",48.862683,2.358685
4,4th (IVe) R,"Hôtel-de-Ville, Paris, FR",48.856426,2.352528


In [53]:
# function to find venues through Foursquare API
def getNearbyVenues(names, latitudes, longitudes, radius=800):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],
#             v['venue']['id'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue',           
                  'Venue Latitude', 
                  'Venue Longitude',
#                   'Venue ID',
                  'Venue Category']
    
    return(nearby_venues)

In [54]:
# finding venues for each neighborhood
venues = getNearbyVenues(names=df2['Neighborhood'],latitudes=df2['lat'],longitudes=df2['lng'])
venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Atocha, Madrid, ES",40.400687,-3.682391,La Cevicucheria,40.402679,-3.680336,Peruvian Restaurant
1,"Atocha, Madrid, ES",40.400687,-3.682391,El Rincón de Fogg,40.404065,-3.677902,Burger Joint
2,"Atocha, Madrid, ES",40.400687,-3.682391,La Polonesa,40.404208,-3.679567,Polish Restaurant
3,"Atocha, Madrid, ES",40.400687,-3.682391,La Neomudéjar,40.404717,-3.683918,Museum
4,"Atocha, Madrid, ES",40.400687,-3.682391,Mesón Restaurante El Segoviano,40.405018,-3.683327,Spanish Restaurant


In [55]:
venues.shape

(1909, 7)

In [56]:
# venue categories in the dataset venues
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 239 uniques categories.


In [57]:
# venue categories in Atocha only
atocha = venues.loc[venues['Neighborhood'] == 'Atocha, Madrid, ES'].copy()
print('There are {} uniques categories.'.format(len(atocha['Venue Category'].unique())))

There are 42 uniques categories.


In [58]:
atocha.shape

(88, 7)

In [59]:
atocha['Venue Category'].unique()

array(['Peruvian Restaurant', 'Burger Joint', 'Polish Restaurant',
       'Museum', 'Spanish Restaurant', 'Art Gallery', 'Bookstore',
       'Sporting Goods Shop', 'Bar', 'Italian Restaurant',
       'Greek Restaurant', 'Farmers Market', 'Asian Restaurant',
       'Restaurant', 'Gym', 'Dessert Shop', 'Mexican Restaurant',
       'Bakery', 'Breakfast Spot', 'Food & Drink Shop', 'Pizza Place',
       'Tapas Restaurant', 'Gymnastics Gym', 'Gift Shop', 'Brewery',
       'Concert Hall', 'Chinese Restaurant', 'Coffee Shop', 'Café',
       'Diner', 'Train Station', 'Hotel', 'Grocery Store',
       'Mediterranean Restaurant', 'Athletics & Sports', 'Platform',
       'Ice Cream Shop', 'Sandwich Place', 'Multiplex',
       'Sushi Restaurant', 'Beer Garden', 'Cafeteria'], dtype=object)

In [60]:
# storing Atocha area coordinates into variables for data visualization
geolocator = Nominatim(user_agent="my-app")
location = geolocator.geocode('Atocha, Madrid, ES')
lat_mad = location.latitude
lng_mad = location.longitude
print(lat_mad,lng_mad)

40.40068665 -3.68239115386921


In [61]:
# visualizing location of venue in a radius of 1000 meters from the coordinates of Atocha area
atocha_map = folium.Map(location=[lat_mad, lng_mad], zoom_start=15)
folium.Circle([lat_mad, lng_mad],
              radius=1000,
              color='red').add_to(atocha_map)

for lat, lon, poi in zip(atocha['Venue Latitude'], atocha['Venue Longitude'], atocha['Venue Category']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lon)],
        # radius=res_count,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(atocha_map)

atocha_map

## One Hot encoding

In [62]:
# one hot encoding
df_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]

df_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Vineyard,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,"Atocha, Madrid, ES",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Atocha, Madrid, ES",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Atocha, Madrid, ES",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Atocha, Madrid, ES",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Atocha, Madrid, ES",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
df_grouped = df_onehot.groupby('Neighborhood').mean().reset_index()
print(df_grouped.shape)
df_grouped

(21, 240)


,Neighborhood,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Vineyard,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,"Atocha, Madrid, ES",0.00,0.00,0.00,0.00,0.00,0.00,0.022727,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,"Batignolles-Monceau, Paris, FR",0.00,0.00,0.00,0.00,0.00,0.00,0.010000,0.02,0.00,...,0.02,0.00,0.00,0.01,0.00,0.06,0.00,0.00,0.00,0.00
2,"Bourse, Paris, FR",0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,...,0.00,0.01,0.00,0.00,0.00,0.06,0.01,0.02,0.00,0.00
3,"Butte-Montmartre, Paris, FR",0.00,0.01,0.00,0.00,0.01,0.01,0.000000,0.00,0.00,...,0.00,0.00,0.02,0.02,0.01,0.02,0.00,0.00,0.00,0.00
4,"Buttes-Chaumont, Paris, FR",0.00,0.00,0.00,0.01,0.00,0.00,0.000000,0.00,0.00,...,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00
5,"Entrepôt, Paris, FR",0.00,0.02,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,...,0.00,0.00,0.02,0.00,0.00,0.02,0.02,0.00,0.00,0.00
6,"Gobelins, Paris, FR",0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,...,0.00,0.00,0.01,0.12,0.00,0.00,0.00,0.00,0.00,0.00
7,"Hôtel-de-Ville, Paris, FR",0.00,0.00,0.01,0.00,0.00,0.00,0.040000,0.02,0.00,...,0.00,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.00
8,"Louvre, Paris, FR",0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.03,0.00,...,0.00,0.02,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00
9,"Luxembourg, Paris, FR",0.00,0.00,0.00,0.01,0.00,0.00,0.020000,0.01,0.00,...,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00


In [35]:
# function to return the most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [64]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_grouped['Neighborhood']

for ind in np.arange(df_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Atocha, Madrid, ES",Spanish Restaurant,Bar,Café,Italian Restaurant,Restaurant,Hotel,Asian Restaurant,Museum,Bakery,Platform
1,"Batignolles-Monceau, Paris, FR",French Restaurant,Hotel,Wine Bar,Italian Restaurant,Bar,Thai Restaurant,Pastry Shop,Theater,Coffee Shop,Korean Restaurant
2,"Bourse, Paris, FR",French Restaurant,Cocktail Bar,Wine Bar,Bistro,Japanese Restaurant,Italian Restaurant,Bakery,Hotel,Creperie,Bar
3,"Butte-Montmartre, Paris, FR",French Restaurant,Bar,Bistro,Pizza Place,Café,Italian Restaurant,Bookstore,Gastropub,Bakery,Park
4,"Buttes-Chaumont, Paris, FR",Bar,French Restaurant,Hotel,Café,Bistro,Restaurant,Supermarket,Pool,Asian Restaurant,Pizza Place
5,"Entrepôt, Paris, FR",Coffee Shop,French Restaurant,Pizza Place,Cocktail Bar,Bistro,Japanese Restaurant,Asian Restaurant,Korean Restaurant,Café,Seafood Restaurant
6,"Gobelins, Paris, FR",Vietnamese Restaurant,Thai Restaurant,Asian Restaurant,French Restaurant,Chinese Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Juice Bar,Bar
7,"Hôtel-de-Ville, Paris, FR",French Restaurant,Ice Cream Shop,Art Gallery,Plaza,Wine Bar,Pub,Park,Hotel,Café,Souvenir Shop
8,"Louvre, Paris, FR",French Restaurant,Hotel,Plaza,Café,Clothing Store,Japanese Restaurant,Restaurant,Historic Site,Cosmetics Shop,Coffee Shop
9,"Luxembourg, Paris, FR",French Restaurant,Italian Restaurant,Hotel,Wine Bar,Bakery,Plaza,Seafood Restaurant,Pastry Shop,Bistro,Sandwich Place


In [74]:
kclusters = 3 

df_grouped_clustering = df_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0])

In [75]:
df_merged = df2

# add clustering labels
df_merged['Cluster Labels'] = kmeans.labels_

# merge df_merged with neighborhoods_venues_sorted dataframe to add latitude/longitude for each neighborhood
df_merged = df_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

df_merged

,District,Neighborhood,lat,lng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arganzuela,"Atocha, Madrid, ES",40.400687,-3.682391,1,Spanish Restaurant,Bar,Café,Italian Restaurant,Restaurant,Hotel,Asian Restaurant,Museum,Bakery,Platform
1,1st (Ie) R,"Louvre, Paris, FR",48.861147,2.338028,0,French Restaurant,Hotel,Plaza,Café,Clothing Store,Japanese Restaurant,Restaurant,Historic Site,Cosmetics Shop,Coffee Shop
2,2nd (IIe) R,"Bourse, Paris, FR",48.867684,2.343126,0,French Restaurant,Cocktail Bar,Wine Bar,Bistro,Japanese Restaurant,Italian Restaurant,Bakery,Hotel,Creperie,Bar
3,3rd (IIIe) R,"Temple, Paris, FR",48.862683,2.358685,0,French Restaurant,Coffee Shop,Cocktail Bar,Café,Clothing Store,Sandwich Place,Wine Bar,Bistro,Burger Joint,Italian Restaurant
4,4th (IVe) R,"Hôtel-de-Ville, Paris, FR",48.856426,2.352528,1,French Restaurant,Ice Cream Shop,Art Gallery,Plaza,Wine Bar,Pub,Park,Hotel,Café,Souvenir Shop
5,5th (Ve) L,"Panthéon, Paris, FR",48.846191,2.346079,0,French Restaurant,Hotel,Bar,Pub,Bakery,Café,Indie Movie Theater,Ice Cream Shop,Plaza,Coffee Shop
6,6th (VIe) L,"Luxembourg, Paris, FR",48.849392,2.332260,0,French Restaurant,Italian Restaurant,Hotel,Wine Bar,Bakery,Plaza,Seafood Restaurant,Pastry Shop,Bistro,Sandwich Place
7,7th (VIIe) L,"Palais-Bourbon, Paris, FR",48.861692,2.319031,0,French Restaurant,Garden,Hotel,Plaza,Bookstore,Historic Site,Art Museum,Italian Restaurant,Fountain,Tea Room
8,8th (VIIIe) R,"Élysée, Paris, FR",48.846644,2.369830,0,French Restaurant,Hotel,Italian Restaurant,Bar,Cocktail Bar,Plaza,Pedestrian Plaza,Bakery,Coffee Shop,Beer Bar
9,9th (IXe) R,"Opéra, Paris, FR",48.870645,2.332330,0,Hotel,French Restaurant,Japanese Restaurant,Italian Restaurant,Gourmet Shop,Pastry Shop,Cocktail Bar,Theater,Dessert Shop,Café


## Results

In [76]:
df_merged.loc[df_merged['Cluster Labels'] == 1]

,District,Neighborhood,lat,lng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arganzuela,"Atocha, Madrid, ES",40.400687,-3.682391,1,Spanish Restaurant,Bar,Café,Italian Restaurant,Restaurant,Hotel,Asian Restaurant,Museum,Bakery,Platform
4,4th (IVe) R,"Hôtel-de-Ville, Paris, FR",48.856426,2.352528,1,French Restaurant,Ice Cream Shop,Art Gallery,Plaza,Wine Bar,Pub,Park,Hotel,Café,Souvenir Shop
17,17th (XVIIe) R,"Batignolles-Monceau, Paris, FR",48.881312,2.315750,1,French Restaurant,Hotel,Wine Bar,Italian Restaurant,Bar,Thai Restaurant,Pastry Shop,Theater,Coffee Shop,Korean Restaurant


In [80]:
# venue categories in Batignolles-Monceau area in Paris only
batignolles = venues.loc[venues['Neighborhood'] == 'Batignolles-Monceau, Paris, FR'].copy()
print('There are {} uniques categories.'.format(len(atocha['Venue Category'].unique())))

There are 42 uniques categories.


In [81]:
batignolles.shape

(100, 7)

In [82]:
batignolles['Venue Category'].unique()

array(['Ice Cream Shop', 'Middle Eastern Restaurant', 'French Restaurant',
       'Creperie', 'Bar', 'Farmers Market', 'Pastry Shop', 'Bagel Shop',
       'History Museum', 'Gym / Fitness Center', 'Wine Bar',
       'Liquor Store', 'Thai Restaurant', 'Pub', 'Café', 'Restaurant',
       'Italian Restaurant', 'Modern European Restaurant',
       'Korean Restaurant', 'Theater', 'BBQ Joint', 'Park', 'Steakhouse',
       'Lebanese Restaurant', 'Hotel', 'Vietnamese Restaurant',
       'Gastropub', 'Bookstore', 'Sushi Restaurant', 'Basque Restaurant',
       'Latin American Restaurant', 'Bakery', 'Japanese Restaurant',
       'Burger Joint', 'Dim Sum Restaurant', 'Coffee Shop',
       'Mediterranean Restaurant', 'Corsican Restaurant',
       'Turkish Restaurant', 'Art Museum', 'Pizza Place', 'Art Gallery',
       'Diner', 'Supermarket', 'Indian Restaurant', 'Bistro',
       'Sandwich Place'], dtype=object)

It seems that Batignolles-Monceau has the same variety of venue categories.

Let's visualize the **location** of each venue on a map of the Batignolles-Monceau area.

In [84]:
# storing Batignolles-Monceau district coordinates into variables for data visualization
geolocator = Nominatim(user_agent="my-app")
location = geolocator.geocode('Batignolles-Monceau, Paris, FR')
lat_ba = location.latitude
lng_ba = location.longitude
print(lat_ba,lng_ba)

48.8813119 2.3157496


In [86]:
# visualizing location of venues in a radius of 1000 meters from the coordinates of Batignolles-Monceau district
batignolles_map = folium.Map(location=[lat_ba, lng_ba], zoom_start=15)
folium.Circle([lat_ba, lng_ba],
              radius=1000,
              color='red').add_to(batignolles_map)

for lat, lon, poi in zip(batignolles['Venue Latitude'], batignolles['Venue Longitude'], batignolles['Venue Category']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lon)],
        # radius=res_count,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(batignolles_map)

batignolles_map